# Diffusion Models

## Basic Concept
- Diffusion models work by gradually adding noise to data and then learning to reverse this process.
- They consist of two main processes:
    - Forward diffusion (adding noise gradually)
    - Reverse diffusion (removing noise gradually)

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp

class DiffusionModel(tf.keras.Model):
    def __init__(self, time_steps=1000, beta_start=1e-4, beta_end=0.02):
        super().__init__()

        #Define noise schedule
        self.time_steps = time_steps
        self.beta = tf.linspace(beta_start, beta_end, time_steps)
        self.alpha = 1 - self.beta
        self.alpha_bar = tf.math.cumprod(self.alpha)

        # Define the U-Net architecture for noise prediction
        self.model = self.build_unet()

    def build_unet(self):
        # Simple U-Net architecture
        inputs = tf.keras.Input(shape=(28, 28, 1))
        x = tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu')(inputs)
        x = tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu')(x)
        x = tf.keras.layers.Conv2D(1, 3, padding='same')(x)
        return tf.keras.Model(inputs=inputs, outputs=x)
    
    def forward_diffusion(self, x_0, t):
        # Add noise according to the noise schedule
        alpha_t = tf.gather(self.alpha_bar, t)
        alpha_t = tf.reshape(alpha_t, (-1, 1, 1, 1))
        
        noise = tf.random.normal(shape=x_0.shape)
        return tf.sqrt(alpha_t) * x_0 + tf.sqrt(1 - alpha_t) * noise, noise
    
    def train_step(self, x_0):
        # Sample random timesteps
        t = tf.random.uniform(
            shape=(tf.shape(x_0)[0],),
            minval=0,
            maxval=self.time_steps,
            dtype=tf.int32
        )
        
        with tf.GradientTape() as tape:
            # Forward diffusion
            x_noisy, noise = self.forward_diffusion(x_0, t)
            
            # Predict noise
            predicted_noise = self.model(x_noisy, training=True)
            
            # Calculate loss
            loss = tf.reduce_mean(tf.square(noise - predicted_noise))
        
        # Update model parameters
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        
        return {'loss': loss}

    def sample(self, n_samples):
        # Start from random noise
        x = tf.random.normal(shape=(n_samples, 28, 28, 1))
        
        # Gradually denoise
        for t in range(self.time_steps - 1, -1, -1):
            t_tensor = tf.constant([t], dtype=tf.int32)
            t_tensor = tf.repeat(t_tensor, n_samples)
            
            # Predict noise
            predicted_noise = self.model(x)
            
            alpha_t = tf.gather(self.alpha, t)
            alpha_bar_t = tf.gather(self.alpha_bar, t)
            
            # Update sample
            if t > 0:
                noise = tf.random.normal(shape=x.shape)
            else:
                noise = 0
                
            x = (1 / tf.sqrt(alpha_t)) * (
                x - (1 - alpha_t) / tf.sqrt(1 - alpha_bar_t) * predicted_noise
            ) + tf.sqrt(self.beta[t]) * noise
            
        return x





Failed to import TF-Keras. Please note that TF-Keras is not installed by default when you install TensorFlow Probability. This is so that JAX-only users do not have to install TensorFlow or TF-Keras. To use TensorFlow Probability with TensorFlow, please install the tf-keras or tf-keras-nightly package.
This can be be done through installing the tensorflow-probability[tf] extra.




ModuleNotFoundError: No module named 'tf_keras'